In [1]:
import cv2
import os
import shutil

# Root folders
original_root = "OriginalImage"
output_root = "ColorVariants"

# Variants
variants = ["Original", "Grayscale", "Swap", "Red", "Green", "Blue"]

# Image size
img_size = (128, 128)

def pad_to_square(img):
    h, w = img.shape[:2]
    max_dim = max(h, w)

    # Amount of padding needed
    top = (max_dim - h) // 2
    bottom = max_dim - h - top
    left = (max_dim - w) // 2
    right = max_dim - w - left

    # Replicate nearest pixel outward
    padded = cv2.copyMakeBorder(
        img,
        top, bottom, left, right,
        borderType = cv2.BORDER_REPLICATE
    )

    return padded

# ---------------------------------------------------------
# 1. Clear output_root entirely, then recreate it
# ---------------------------------------------------------
if os.path.exists(output_root):
    shutil.rmtree(output_root)

for v in variants:
    os.makedirs(os.path.join(output_root, v), exist_ok=True)

# ---------------------------------------------------------
# 2. Process per variant
# ---------------------------------------------------------
for variant in variants:
    variant_path = os.path.join(output_root, variant)

    for class_name in os.listdir(original_root):
        class_src = os.path.join(original_root, class_name)
        class_dst = os.path.join(variant_path, class_name)
        os.makedirs(class_dst, exist_ok=True)

        # Get sorted list of images to ensure consistent first 50 selection
        images = sorted(os.listdir(class_src))
        images = images[:50]   # Take only the first 50

        for img_name in images:
            img_path = os.path.join(class_src, img_name)
            img = cv2.imread(img_path)

            # ----------- Square padding then resizing -----------
            squared = pad_to_square(img)
            img_resized = cv2.resize(squared, img_size)

            # ---------------- Variant Conversions ---------------- #

            if variant == "Original":
                converted = img_resized

            elif variant == "Grayscale":
                gray = cv2.cvtColor(img_resized, cv2.COLOR_BGR2GRAY)
                converted = cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)

            elif variant == "Swap":
                b, g, r = cv2.split(img_resized)
                converted = cv2.merge([r, g, b])

            elif variant == "Red":
                b, g, r = cv2.split(img_resized)
                converted = cv2.merge([b*0, g*0, r])

            elif variant == "Green":
                b, g, r = cv2.split(img_resized)
                converted = cv2.merge([b*0, g, r*0])

            elif variant == "Blue":
                b, g, r = cv2.split(img_resized)
                converted = cv2.merge([b, g*0, r*0])

            # ----------------------------------------------------- #

            out_path = os.path.join(class_dst, img_name)
            cv2.imwrite(out_path, converted)


In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

aug = ImageDataGenerator(
    rotation_range = 20,
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    shear_range = 0.15,
    zoom_range = 0.15,
    horizontal_flip = True,
    fill_mode = 'nearest'
)

train_datagen = ImageDataGenerator(
    validation_split=0.2,  # 80/20 split
    **{k: v for k, v in aug.__dict__.items() if k in ["rotation_range", "width_shift_range",
                                                      "height_shift_range", "shear_range",
                                                      "zoom_range", "horizontal_flip",
                                                      "fill_mode"]}
)

batch_size = 32

train_gen = train_datagen.flow_from_directory(
    original_root,
    target_size = img_size,
    class_mode = "categorical",
    batch_size = batch_size,
    subset = "training"
)

val_gen = train_datagen.flow_from_directory(
    original_root,
    target_size = img_size,
    class_mode = "categorical",
    batch_size = batch_size,
    subset = "validation"
)


Found 2560 images belonging to 16 classes.
Found 640 images belonging to 16 classes.


In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

def build_model(num_classes):
    model = Sequential([
        Conv2D(32, (3, 3), activation = 'relu', input_shape = (128, 128, 3)),
        MaxPooling2D(2, 2),

        Conv2D(64, (3, 3), activation = 'relu'),
        MaxPooling2D(2, 2),

        Conv2D(128, (3, 3), activation = 'relu'),
        MaxPooling2D(2, 2),

        Flatten(),
        Dense(128, activation = 'relu'),
        Dropout(0.4),
        Dense(num_classes, activation = 'softmax')
    ])

    model.compile(
        optimizer = "adam",
        loss = "categorical_crossentropy",
        metrics = ["accuracy"]
    )
    return model

In [4]:
import os

variants = [
    "ColorVariants/Original",
    "ColorVariants/Grayscale",
    "ColorVariants/Swap",
    "ColorVariants/Red",
    "ColorVariants/Green",
    "ColorVariants/Blue"
]

for variant_path in variants:
    train_gen = train_datagen.flow_from_directory(
        variant_path,
        target_size=img_size,
        class_mode="categorical",
        batch_size=batch_size,
        subset="training"
    )
    val_gen = train_datagen.flow_from_directory(
        variant_path,
        target_size=img_size,
        class_mode="categorical",
        batch_size=batch_size,
        subset="validation"
    )

    model = build_model(num_classes = train_gen.num_classes)

    history = model.fit(
        train_gen,
        validation_data = val_gen,
        epochs = 20
    )

    model.save(f"cnn_{os.path.basename(variant_path)}.keras")

Found 640 images belonging to 16 classes.
Found 160 images belonging to 16 classes.


c:\Users\user\anaconda3\lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\user\anaconda3\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 23s 1s/step - accuracy: 0.0541 - loss: 75.1567 - val_accuracy: 0.1875 - val_loss: 2.4823
Epoch 2/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 8s 406ms/step - accuracy: 0.1728 - loss: 2.5444 - val_accuracy: 0.2000 - val_loss: 2.4348
Epoch 3/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 9s 448ms/step - accuracy: 0.2150 - loss: 2.3586 - val_accuracy: 0.2750 - val_loss: 2.0928
Epoch 4/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 9s 472ms/step - accuracy: 0.2886 - loss: 2.1171 - val_accuracy: 0.2125 - val_loss: 2.4311
Epoch 5/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 9s 458ms/step - accuracy: 0.2271 - loss: 2.3308 - val_accuracy: 0.3500 - val_loss: 2.0257
Epoch 6/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 8s 393ms/step - accuracy: 0.3134 - loss: 2.0650 - val_accuracy: 0.3000 - val_loss: 2.0844
Epoch 7/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 9s 447ms/step - accuracy: 0.2746 - loss: 2.4447 - val_accuracy: 0.2937 - val_loss: 2.1013
Epoch 8/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 9s 447ms/step - accuracy: 0.3656 - loss: 1.9048 - val_accuracy: 0.3